# High Dimensional Weighted Gene Correlation Network Analysis (hdWGCNA)

**Authorship:**
Adam Klie, *08/24/2022*
***
**Description:**
Notebook to run a WGCNA on stimulated pancreatic islet multiome data (well just the RNA)
***

# Load packages and data

In [2]:
# single-cell analysis package
library(Seurat)

# plotting and data science packages
library(tidyverse)
library(cowplot)
library(patchwork)

# co-expression network analysis packages:
library(WGCNA)
library(hdWGCNA)

# using the cowplot theme for ggplot
theme_set(theme_cowplot())

Attaching SeuratObject

Attaching sp

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots


Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




In [1]:
# Parameters
CELL.TYPES <- "beta"  # Meaning α, β, δ in this context
ASSAY <- "RNA"
NORMALIZATION <- "ND"
NN <- 25
GENES <- "all"

In [3]:
# File names for saving
NAME=paste0(CELL.TYPES, "cells_", GENES, "genes_", NORMALIZATION, "_", NN, "neighbors")
OUT= file.path("./module_trait_correlation", NAME)
OUT

[1] "./module_trait_correlation/betacells_allgenes_ND_25neighbors"

In [4]:
# load the preprocessed RDS from 1_network_construction.ipynb
seurat_obj <- readRDS(sprintf('network_construction/%s_hdWGCNA.rds', NAME))

# Trait correlations

## Set-up columns in meta data for plotting

In [5]:
# Make the timepoint var an int
seurat_obj$time_point <- as.integer(seurat_obj$time_point)

Loading required package: Signac



In [6]:
# Add in binary comparison columns for treatments
seurat_obj@meta.data <- cbind(
    seurat_obj@meta.data, 
    binarizeCategoricalVariable(
        seurat_obj$condition, 
        levelOrder = c("ctrl", "palmitate"),
    )
)
colnames(seurat_obj@meta.data)

[1] "orig.ident"               "nCount_RNA"              
 [3] "nFeature_RNA"             "percent.mt"              
 [5] "nCount_RNA_raw"           "nFeature_RNA_raw"        
 [7] "nCount_ATAC"              "nFeature_ATAC"           
 [9] "nCount_SCT"               "nFeature_SCT"            
[11] "SCT.weight"               "ATAC.weight"             
[13] "wsnn_res.0.2"             "seurat_clusters"         
[15] "log_nCount_ATAC"          "log_nCount_SCT"          
[17] "log_nFeature_ATAC"        "log_nFeature_SCT"        
[19] "nCount_mpeak"             "nFeature_mpeak"          
[21] "batch"                    "condition"               
[23] "time_point"               "sample"                  
[25] "atac_raw_reads"           "atac_mitochondrial_reads"
[27] "atac_fragments"           "atac_pct.mt"             
[29] "TSS.enrichment"           "TSS.percentile"          
[31] "mpeak.weight"             "wsnn_res.0.3"            
[33] "nCount_peaks"             "nFeature_peaks"          
[35] "peaks.weight"             "wsnn_res.0.5"            
[37] "nCount_peak"              "nFeature_peak"           
[39] "cell.type.1"              "cell.type.2"             
[41] "metacell_grouping"        "palmitate.vs.ctrl"

In [7]:
# Double check these, they should match up. Note that untreated is equivalent to 1 here
print(table(seurat_obj$condition))
print(table(seurat_obj$palmitate.vs.ctrl))


     ctrl palmitate 
     3533      3509 

   0    1 
3533 3509 


In [8]:
print(table(seurat_obj$time_point))


   0    6   24   48   72 
1070 2168 1945  491 1368 


In [9]:
# Add in binary comparison columns for cell types (one vs rest)
seurat_obj@meta.data <- cbind(seurat_obj@meta.data, 
                              binarizeCategoricalColumns.forPlots(seurat_obj$cell.type.2))
colnames(seurat_obj@meta.data)

[1] "orig.ident"               "nCount_RNA"              
 [3] "nFeature_RNA"             "percent.mt"              
 [5] "nCount_RNA_raw"           "nFeature_RNA_raw"        
 [7] "nCount_ATAC"              "nFeature_ATAC"           
 [9] "nCount_SCT"               "nFeature_SCT"            
[11] "SCT.weight"               "ATAC.weight"             
[13] "wsnn_res.0.2"             "seurat_clusters"         
[15] "log_nCount_ATAC"          "log_nCount_SCT"          
[17] "log_nFeature_ATAC"        "log_nFeature_SCT"        
[19] "nCount_mpeak"             "nFeature_mpeak"          
[21] "batch"                    "condition"               
[23] "time_point"               "sample"                  
[25] "atac_raw_reads"           "atac_mitochondrial_reads"
[27] "atac_fragments"           "atac_pct.mt"             
[29] "TSS.enrichment"           "TSS.percentile"          
[31] "mpeak.weight"             "wsnn_res.0.3"            
[33] "nCount_peaks"             "nFeature_peaks"          
[35] "peaks.weight"             "wsnn_res.0.5"            
[37] "nCount_peak"              "nFeature_peak"           
[39] "cell.type.1"              "cell.type.2"             
[41] "metacell_grouping"        "palmitate.vs.ctrl"       
[43] "data.SC.beta.1"           "data.SC.beta.2"          
[45] "data.SC.beta.3"

In [15]:
# Double check
#print(table(seurat_obj$cell.type.2, seurat_obj$data.SC.alpha))
print(table(seurat_obj$cell.type.2, seurat_obj$data.SC.beta.1))
print(table(seurat_obj$cell.type.2, seurat_obj$data.SC.beta.2))
print(table(seurat_obj$cell.type.2, seurat_obj$data.SC.beta.3))
#print(table(seurat_obj$cell.type.2, seurat_obj$data.SC.delta))
#print(table(seurat_obj$cell.type.2, seurat_obj$data.SC.EC))

           
               0    1
  SC.beta.1    0 3124
  SC.beta.2 3080    0
  SC.beta.3  838    0
           
               0    1
  SC.beta.1 3124    0
  SC.beta.2    0 3080
  SC.beta.3  838    0
           
               0    1
  SC.beta.1 3124    0
  SC.beta.2 3080    0
  SC.beta.3    0  838


## Plot correlation heatmaps

### Cell type correlations

In [16]:
# list of traits to correlate for cell type
cur_traits <- c('data.SC.beta.1', 'data.SC.beta.2', 'data.SC.beta.3')
seurat_obj <- ModuleTraitCorrelation(
  seurat_obj,
  features="MEs",
  traits = cur_traits,
)

In [17]:
# Grabe the correlation table and eventually save it
mt_cor <- GetModuleTraitCorrelation(seurat_obj)
print(names(mt_cor))
head(mt_cor$cor$all_cells)

[1] "cor"  "pval" "fdr" 


,betacells_allgenes_ND_25neighbors-M1,betacells_allgenes_ND_25neighbors-M2,betacells_allgenes_ND_25neighbors-M3,betacells_allgenes_ND_25neighbors-M4,betacells_allgenes_ND_25neighbors-M5
data.SC.beta.1,-0.6203667,-0.7627454,0.69219470,0.16253332,0.09938363
data.SC.beta.2,0.4314970,0.7501051,-0.63974844,-0.12977087,-0.18701188
data.SC.beta.3,0.2907939,0.0211298,-0.08195155,-0.05056983,0.13402154


In [19]:
# Grab and order the correlation dataframe to match the built-in function ordering
corr_df <- mt_cor$cor$all_cells
corr_df <- corr_df[c('data.SC.beta.1', 'data.SC.beta.2', 'data.SC.beta.3'), 1:5]

In [20]:
# Display the correlation values within a heatmap plot
options(repr.plot.width=16, repr.plot.height=8)
png(sprintf("%s_ModuleCelltypeCorrelationsAllCells.png", OUT), widt=1200, height=800)
par(mar=c(10,6,4,1))
textMatrix = paste(
    signif(corr_df, 2), sep="")
dim(textMatrix) = dim(corr_df)
labeledHeatmap(
    Matrix=corr_df,
    xLabels=colnames(corr_df),
    yLabels=rownames(corr_df),
    ySymbols=rownames(corr_df),
    textMatrix=textMatrix,
    colorLabels=FALSE,
    colors=blueWhiteRed(50),
    cex.text=1.2,
    cex.lab.x=1,
    cex.lab.y=1,
    zlim=c(-1, 1),
    main=paste("Module-Cell type relationships"),
)
dev.off()

png 
  2

In [21]:
# Plot a heatmap of p-values for correlations across cell-types
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleCelltypeCorrelationFDRValues.png", OUT), widt=1000, height=1200)
PlotModuleTraitCorrelation(
  seurat_obj,
  label = 'fdr',
  label_symbol = 'numeric',
  text_size = 5,
  text_digits = 3,
  combine=TRUE,
  plot_max = 0.4
)
dev.off()

[1] "all_cells"
[1] "data.SC.beta.1" "data.SC.beta.2" "data.SC.beta.3"
[1] "SC.beta"
[1] "data.SC.beta.1" "data.SC.beta.2" "data.SC.beta.3"


png 
  2

In [22]:
options(repr.plot.width=12, repr.plot.height=12)
png(sprintf("%s_ModuleCelltypeCorrelationFDRStars.png", OUT), widt=1000, height=1200)
PlotModuleTraitCorrelation(
  seurat_obj,
  label = 'fdr',
  label_symbol = 'stars',
  text_size = 5,
  text_digits = 3,
  combine=TRUE,
  plot_max = 0.4
)
dev.off()

[1] "all_cells"
[1] "data.SC.beta.1" "data.SC.beta.2" "data.SC.beta.3"
[1] "SC.beta"
[1] "data.SC.beta.1" "data.SC.beta.2" "data.SC.beta.3"


png 
  2

### Treatment correlations

In [23]:
# list of traits to correlate
cur_traits <- c('palmitate.vs.ctrl')

seurat_obj <- ModuleTraitCorrelation(
  seurat_obj,
  traits = cur_traits,
  features="MEs",
  group.by='cell.type.2'
)

In [24]:
# Grabe the correlation table and eventually save it
mt_cor <- GetModuleTraitCorrelation(seurat_obj)
print(names(mt_cor))
head(mt_cor$cor$all_cells)

[1] "cor"  "pval" "fdr" 


betacells_allgenes_ND_25neighbors-M1 betacells_allgenes_ND_25neighbors-M2 
                         0.008303432                          0.020129391 
betacells_allgenes_ND_25neighbors-M3 betacells_allgenes_ND_25neighbors-M4 
                         0.053283008                          0.150403223 
betacells_allgenes_ND_25neighbors-M5 
                         0.071012540

In [25]:
# Grab and order the correlation dataframe to match the built-in function ordering
corr_df <- do.call(rbind, mt_cor$cor)
corr_df

,betacells_allgenes_ND_25neighbors-M1,betacells_allgenes_ND_25neighbors-M2,betacells_allgenes_ND_25neighbors-M3,betacells_allgenes_ND_25neighbors-M4,betacells_allgenes_ND_25neighbors-M5
all_cells,0.008303432,0.02012939,0.05328301,0.1504032,0.07101254
SC.beta.1,0.024105396,0.01565970,0.09144104,0.1802153,0.09044621
SC.beta.2,0.036196460,0.09011647,0.02676081,0.1330350,0.05861573
SC.beta.3,-0.033992044,0.03856591,0.05260006,0.1118299,0.04838356


In [26]:
# Display the correlation values within a heatmap plot
options(repr.plot.width=16, repr.plot.height=8)
png(sprintf("%s_ModuleTreatmentCorrelationsByCellType.png", OUT), widt=1200, height=800)
par(mar=c(10,6,4,1))
textMatrix = paste(
    signif(corr_df, 2), sep="")
dim(textMatrix) = dim(corr_df)
labeledHeatmap(
    Matrix=corr_df,
    xLabels=colnames(corr_df),
    yLabels=rownames(corr_df),
    ySymbols=rownames(corr_df),
    textMatrix=textMatrix,
    colorLabels=FALSE,
    colors=blueWhiteRed(50),
    cex.text=1.2,
    cex.lab.x=1,
    cex.lab.y=1,
    zlim=c(-1, 1),
    main=paste("Module-palmitate treatment relationships in different cell types"),
)
dev.off()

png 
  2

In [27]:
# Grab and order the correlation dataframe to match the built-in function ordering
pval_df <- do.call(rbind, mt_cor$pval)
pval_df

,betacells_allgenes_ND_25neighbors-M1,betacells_allgenes_ND_25neighbors-M2,betacells_allgenes_ND_25neighbors-M3,betacells_allgenes_ND_25neighbors-M4,betacells_allgenes_ND_25neighbors-M5
all_cells,0.48600038,9.120759e-02,7.687240e-06,0.000000e+00,2.437265e-09
SC.beta.1,0.17798876,3.815919e-01,3.062262e-07,0.000000e+00,4.113140e-07
SC.beta.2,0.04457269,5.459269e-07,1.375895e-01,1.234568e-13,1.135847e-03
SC.beta.3,0.32569371,2.647805e-01,1.281447e-01,1.184357e-03,1.617088e-01


In [31]:
# Display the correlation values within a heatmap plot
options(repr.plot.width=16, repr.plot.height=8)
png(sprintf("%s_ModuleTreatmentPValsByCellType.png", OUT), widt=1200, height=800)
par(mar=c(10,6,4,1))
textMatrix = paste(
    signif(pval_df, 2), sep="")
dim(textMatrix) = dim(pval_df)
labeledHeatmap(
    Matrix=pval_df,
    xLabels=colnames(pval_df),
    yLabels=rownames(pval_df),
    ySymbols=rownames(pval_df),
    textMatrix=textMatrix,
    colorLabels=FALSE,
    cex.text=1.2,
    cex.lab.x=1,
    cex.lab.y=1,
    zlim=c(0, 1),
    main=paste("Module-palmitate treatment relationships in different cell types"),
)
dev.off()

png 
  2

## Timepoint correlations

In [33]:
# list of traits to correlate
cur_traits <- c('time_point')

seurat_obj <- ModuleTraitCorrelation(
  seurat_obj,
  traits = cur_traits,
  features="MEs",
  group.by='cell.type.2'
)

In [34]:
# Grabe the correlation table and eventually save it
mt_cor <- GetModuleTraitCorrelation(seurat_obj)
print(names(mt_cor))
head(mt_cor$cor$all_cells)

[1] "cor"  "pval" "fdr" 


betacells_allgenes_ND_25neighbors-M1 betacells_allgenes_ND_25neighbors-M2 
                         0.001195445                          0.045725287 
betacells_allgenes_ND_25neighbors-M3 betacells_allgenes_ND_25neighbors-M4 
                        -0.022262654                         -0.053026564 
betacells_allgenes_ND_25neighbors-M5 
                        -0.087852857

In [35]:
# Grab and order the correlation dataframe to match the built-in function ordering
corr_df <- do.call(rbind, mt_cor$cor)
corr_df

,betacells_allgenes_ND_25neighbors-M1,betacells_allgenes_ND_25neighbors-M2,betacells_allgenes_ND_25neighbors-M3,betacells_allgenes_ND_25neighbors-M4,betacells_allgenes_ND_25neighbors-M5
all_cells,0.001195445,0.045725287,-0.022262654,-0.05302656,-0.08785286
SC.beta.1,-0.040450542,-0.006445666,-0.006042898,-0.09311255,-0.10256027
SC.beta.2,-0.053598573,0.021707564,0.044283025,-0.02011034,-0.06812608
SC.beta.3,-0.008713722,-0.004258160,0.067248457,0.02694036,-0.08668774


In [36]:
# Display the correlation values within a heatmap plot
options(repr.plot.width=16, repr.plot.height=8)
png(sprintf("%s_ModuleTimePointCorrelationsByCellType.png", OUT), widt=1200, height=800)
par(mar=c(10,6,4,1))
textMatrix = paste(
    signif(corr_df, 2), sep="")
dim(textMatrix) = dim(corr_df)
labeledHeatmap(
    Matrix=corr_df,
    xLabels=colnames(corr_df),
    yLabels=rownames(corr_df),
    ySymbols=rownames(corr_df),
    textMatrix=textMatrix,
    colorLabels=FALSE,
    colors=blueWhiteRed(50),
    cex.text=1.2,
    cex.lab.x=1,
    cex.lab.y=1,
    zlim=c(-1, 1),
    main=paste("Module-time point relationships in different cell types"),
)
dev.off()

png 
  2

In [37]:
# Grab and order the correlation dataframe to match the built-in function ordering
pval_df <- do.call(rbind, mt_cor$pval)
pval_df

,betacells_allgenes_ND_25neighbors-M1,betacells_allgenes_ND_25neighbors-M2,betacells_allgenes_ND_25neighbors-M3,betacells_allgenes_ND_25neighbors-M4,betacells_allgenes_ND_25neighbors-M5
all_cells,0.920106195,0.0001238194,0.06174721,8.501369e-06,1.521006e-13
SC.beta.1,0.023764661,0.7187532010,0.73564795,1.852473e-07,9.179236e-09
SC.beta.2,0.002924813,0.2284451408,0.01397847,2.645342e-01,1.545107e-04
SC.beta.3,0.801137622,0.9020415598,0.05165219,4.360664e-01,1.205815e-02


In [38]:
# Display the correlation values within a heatmap plot
options(repr.plot.width=16, repr.plot.height=8)
png(sprintf("%s_ModuleTimePointPValsByCellType.png", OUT), widt=1200, height=800)
par(mar=c(10,6,4,1))
textMatrix = paste(
    signif(pval_df, 2), sep="")
dim(textMatrix) = dim(pval_df)
labeledHeatmap(
    Matrix=pval_df,
    xLabels=colnames(pval_df),
    yLabels=rownames(pval_df),
    ySymbols=rownames(pval_df),
    textMatrix=textMatrix,
    colorLabels=FALSE,
    cex.text=1.2,
    cex.lab.x=1,
    cex.lab.y=1,
    zlim=c(0, 1),
    main=paste("Module-time point relationships in different cell types"),
)
dev.off()

png 
  2

---

# Scratch